In [1]:
import numpy as np
from math import *

from tensorflow import keras
import os

from PIL import Image
import progressbar

import shutil

In [2]:
print("Starting Dir {}".format(os.getcwd()))
os.mkdir('affineMNIST')
os.chdir('affineMNIST')
[os.mkdir(str(classes)) for classes in range(10)]
print("Current Dir {}".format(os.getcwd()))

Starting Dir /content
Current Dir /content/affineMNIST


In [3]:
(xTrain, yTrain), (xTest, yTest) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
def augmentDimsBy(inc, image):
  dim = image.shape[0]
  paddedImage = image
  count = 0
  for i in range(2*inc):
    if i%2 == 0:
      paddedImage = np.append(paddedImage, np.zeros((dim + i%2 + count, 1)), axis = 1)
      paddedImage = np.rot90(paddedImage)
      
      paddedImage = np.append(paddedImage, np.zeros((dim + (i+1)%2 + count,1)), axis = 1)
      paddedImage = np.rot90(paddedImage)

      count += 1
  
  if inc %2 != 0:
    paddedImage = np.rot90(paddedImage, k = 2)

  return paddedImage

In [5]:
def generatePointsAndValuesOfInterest(img):
  pointsAndValues = np.reshape([0,0,0], (1,3))
  dim = img.shape[0]
  for i in range(dim):
    for j in range(dim):
      if img[i,j] > 0.0:
        pointAndValue = np.reshape([i,j,img[i,j]],(1,3))
        pointsAndValues = np.concatenate([pointsAndValues, pointAndValue])

  pointsAndValues = np.delete(pointsAndValues,[0], axis = 0).T

  return pointsAndValues

In [6]:
def transformPoints(points,parameters = np.eye(3)[:-1].T):
  affinePoints = np.matmul(parameters,points)
  
  affineX, affineY = np.append([],affinePoints[0]), np.append([],affinePoints[1])
  affineX, affineY = np.append(affineX,affineX[0]), np.append(affineY,affineY[0])

  return affineX,affineY, affinePoints

In [7]:
def interpolateForPoints(x,y):
  xFloor, yFloor = floor(x), floor(y)
  xFract, yFract = x - xFloor,y - yFloor

  setOfPoints = {(xFloor, yFloor)}

  if False == (xFract == 0.0 and yFract == 0.0):
    if xFract > 0.0:
      setOfPoints.add((round(x),yFloor))
    if yFract > 0.0:
      setOfPoints.add((xFloor,round(y)))
    if xFract > 0.0 and yFract > 0.0:
      setOfPoints.add((round(x),round(y)))

  return setOfPoints

In [8]:
def produceAffineImage(dim,affinePointsAndValues):
  affineImage = np.zeros((dim,dim))
  indexForValue = 0
  affineValues = affinePointsAndValues[-1:]
  affineXs = affinePointsAndValues[0:1][0]
  affineYs = affinePointsAndValues[1:2][0]
  values = np.nditer(affineValues)

  for value in values:
  ### interpolation with equal distribution to smoothen the image
    interplolatedPoints = interpolateForPoints(affineXs[indexForValue], affineYs[indexForValue])
    for (i,j) in interplolatedPoints:
      if i >= 0 and j >= 0 and i <dim and j < dim:
        affineImage[int(i),int(j)] = value  # values are overwritten if they repeat across values

    indexForValue += 1

  return affineImage

In [9]:
def writeImage(img, filename):
  pillowImage = Image.fromarray(img)
  pillowImage = pillowImage.convert("L")
  pillowImage.save(filename)

In [10]:
def zipDir():
  os.chdir('/content')
  print("File path selected {}".format(os.getcwd()))
  print("Archiving images in progress...")
  #shutil.make_archive('affineMNIST', 'zip', '/content/affineMNIST')
  shutil.make_archive(base_name= 'affineMNIST', format='zip', root_dir= 'affineMNIST')
  print("Archiving completed")

In [11]:
def generateDataset():
  index = 0
  widgets = ['Saving dataset : ', progressbar.AnimatedMarker()] 
  bar = progressbar.ProgressBar(widgets=widgets)
  
  indices = yTrain.size
  for img, label in zip(xTrain, yTrain):
    paddedImage = augmentDimsBy(21, img)
    identityParams = np.eye(3)[:-1]
    affineParams = identityParams + (np.random.random(6).reshape(2,-1) - 0.5)/3.5
    pointsAndValues = generatePointsAndValuesOfInterest(paddedImage)
    points = np.concatenate([pointsAndValues[:1], pointsAndValues[1:2], np.ones_like(pointsAndValues[:1])], axis = 0)
    _,_,affinePoints = transformPoints(points,affineParams)
    affinePointsAndValues = np.concatenate([affinePoints,pointsAndValues[-1:]])
    
    affineImage = produceAffineImage(dim = paddedImage.shape[0],affinePointsAndValues = affinePointsAndValues)
    filename = str(label) + '_' + str(index) + '.png'

    os.chdir(str(label))
    writeImage(affineImage,filename)
    os.chdir('../')
    bar.update(float(index)/indices)
    index += 1

  zipDir()

In [12]:
generateDataset()

Saving dataset : /                                                             

File path selected /content
Archiving images in progress...
Archiving completed
